In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import os
import numpy as np
import datetime
import glob
from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score
from pathlib import Path

DOWNLOAD_DIR = '<your-path-here>'



In [2]:
def read_single_csv_entso_e(file):
    return pd.read_csv(file, sep='\t', encoding='utf-16')


def load_complete_entso_e_data(directory):

    pattern = Path(directory) / '*.csv'
    files = glob.glob(str(pattern))
    
    print(f'Concatenating {len(files)} csv files...')
    
    each_csv_file = [read_single_csv_entso_e(file) for file in files]
    data = pd.concat(each_csv_file, ignore_index=True)

    data["DateTime"] = pd.to_datetime(data["DateTime"])

    data = data.sort_values(by=["AreaName", "DateTime"])

    return data


power_demand = load_complete_entso_e_data(DOWNLOAD_DIR)

What is in there?

In [3]:
power_demand

Hm... that's a lot of information. Let's reduce it...

In [4]:
power_demand.columns

Which countries and regions are there?

In [5]:
power_demand["AreaName"].unique()

puh... a lot! Is Austria there?

In [6]:
"Austria" in power_demand["AreaName"].values

Let's get Austrian data then and...

In [7]:
def get_country_data(data, country):
    ret_data = data[data["AreaName"] == country]
    ret_data.index = ret_data["DateTime"]
    return ret_data


power_demand_at_mw = get_country_data(power_demand, "Austria")

## Exercise 1

Let's do sum summary statistics. Calculate mean, standard deviation, min, max and the 25%, 50% and 75% quantile of the distribution of the load column. Hint: There may be a single pandas function that does it for you...

In a second step, do the same for Germany. Does the *10 rule hold? (everything in Germany is ten times as big as in Austria)

Now let's plot Austrian data.

In [8]:
def plot_data(data, xlabel="Time"):
    
    data["TotalLoadValue"].plot()
    plt.ylabel("Average electricity load (MW)")
    plt.xlabel(xlabel)


plot_data(power_demand_at_mw, "Electricity load (MWh)")

Hm... there may be outliers. But how complete is Austrian data?

In [9]:
def check_data_completeness(data):
    number_of_nas = np.sum(pd.isna(data["TotalLoadValue"]))
    number_of_0s = np.sum(data["TotalLoadValue"] == 0)

    print(f'The data contains {number_of_nas} NAs and {number_of_0s} zeros')


check_data_completeness(power_demand_at_mw)

How complete is e.g. North Macedonian data in comparison?

In [10]:
power_demand_nm_mw = get_country_data(power_demand, "North Macedonia")
check_data_completeness(power_demand_nm_mw)

## Exercise 2

What is the time resolution of the dataset? There are several ways of finding it, try to find at least two.

Let's aggregate the data to hourly data. 

In [11]:
power_demand_at_hourly_mw = power_demand_at_mw.resample('1h').mean()

In [12]:
power_demand_at_hourly_mw

In [13]:
plot_data(power_demand_at_hourly_mw)


Let's look into the data more closely and find some regularities...

In [14]:
power_demand_at_yearly_mw = power_demand_at_hourly_mw.resample('1y').mean()

In [15]:
power_demand_at_yearly_mw

In [16]:
plot_data(power_demand_at_yearly_mw)

Hm... 2014 and 2020 are incomplete. We should select the correct period therefore...

In [17]:
power_demand_at_yearly_full_years_mw = power_demand_at_yearly_mw['2015-01-01':'2019-12-31']

plot_data(power_demand_at_yearly_full_years_mw)

Hm... hard to tell if there is a trend. Time-series too short. Let's neglect it for the moment.

In [18]:
power_demand_at_monthly_mw = power_demand_at_hourly_mw.resample('1m').mean()
power_demand_at_monthly_mw = power_demand_at_monthly_mw['2015-01-01':'2019-12-31']

plot_data(power_demand_at_monthly_mw)

hm... seems to be seasonal, right? But let's also select the correct period to show full years only.

To understand it better, we could simply take the monthly average...

In [19]:
power_demand_at_monthly_mean_mw = power_demand_at_monthly_mw.groupby(power_demand_at_monthly_mw.index.month).mean()

plot_data(power_demand_at_monthly_mean_mw, "Month of Year")

Hm... power demand seems to be seasonal.


## Exercise 3

There are at least two other time scales over which data shows seasonality. Can you find them and plot them? Hint: ```power_demand_at_hourly_mw.index.weekday``` may be very useful here.

Let's use all of what we learnt to predict demand! For the moment, we just use linear regression.

Our predicted value Y is the hourly demand.

In [20]:
# Our predicted variable Y is the hourly load

power_demand_at_hourly_reduced_mwh = select_period(power_demand_at_hourly_mwh,
                                                   2015, 1, 1, 2019, 12, 31)

Y = np.array(power_demand_at_hourly_reduced_mwh["TotalLoadValue"]).reshape(-1, 1)

In [ ]:
Y

Our predictor variables (or features) are simply months, weekdays, and hours. 

### Exercise 4

How can this be modelled? Who knows what a dummy is? How are they correctly used?

In [ ]:
monthly_dummies = np.array(pd.get_dummies(
    power_demand_at_hourly_reduced_mwh.DateTime.dt.month))[:, :11]

weekday = np.array(pd.get_dummies(
    power_demand_at_hourly_reduced_mwh.DateTime.dt.weekday))[:, :6]

hour = np.array(pd.get_dummies(
    power_demand_at_hourly_reduced_mwh.DateTime.dt.hour))[:, :23]

In [ ]:
monthly_dummies

Joining them all together.

In [ ]:
X = np.append(monthly_dummies, weekday, axis=1)
X = np.append(X, hour, axis=1)

In [ ]:
X.shape

Let's put this into a linear model.

In [ ]:
# regr = linear_model.LinearRegression(fit_intercept = False)
regr = linear_model.LinearRegression()

# Train the model using the training sets
regr.fit(X, Y)

# The coefficients
print('Coefficients: \n', regr.coef_)


What does the intercept of the regression tell us?

In [ ]:
regr.intercept_

Let's predict with the model on the training data...

In [ ]:
pred = regr.predict(X)

How does it look like?

In [ ]:
plt.plot(Y[1000:1100, :])
plt.plot(pred[1000:1100, :], alpha=0.7)

Pretty cool for such a simple model. How does it look over the whole range?

In [ ]:
plt.plot(Y)
plt.plot(pred, alpha=0.5)

And R^2?

In [ ]:
r2_score(Y, pred)

Wowh. This is pretty high for such a simple model. It means we have rather regular data.

## Exercise 5

Before you do the exercise, let's vote. What expectation do you have when comparing the mean of Y and of pred?

Now, calculate the mean of the two values.

## Exercise 6

Now let's work on the results of the regression. What do coefficients tell us? 

Now plot the dummies for months, weekdays and hours seperately.


In [ ]:
coefs = np.array(regr.coef_).T

# months

In [ ]:
# weekdays

In [ ]:
# hours

## Exercise 7

How do residuals look like now, i.e. the difference between actual value and prediction? 

Plot the residuals.